In [1]:
# 기본 세팅

from urllib.request import Request, urlopen
from urllib.parse import quote, urlencode, quote_plus, unquote
import json
from bs4 import BeautifulSoup as bs

import copy
import os
import datetime as dt
import time as t
import numpy as np
import pandas as pd
import random as rd

from tqdm.notebook import tqdm as tq

# 날짜 지정

In [2]:
'''1. 목표 날짜 지정  (검색할 날짜)'''
# 검색 시작 날짜    
first_date = 20200201
current_date = (int(str(first_date)[:4]) , int(str(first_date)[4:6]), int(str(first_date)[6:]))

# 검색 종료 날짜
end_date = 20200205

# 크롤링

In [3]:
########################################
##### 1. url 크롤링                #####
########################################


##############
data_set = {}          # 데이터가 담길 dictionary
############## 


# first_date 에서 날짜 넘겨받는 부분  (시작점)
current_date = (int(str(first_date)[:4]) , int(str(first_date)[4:6]), int(str(first_date)[6:]))
current_date_WHILE = int(dt.datetime(current_date[0], current_date[1], current_date[2]).strftime('%Y%m%d'))


'''1. 날짜 하루씩 추가하면서 크롤링 진행'''
# '날짜 지정'에서 설정한 검색종료 날짜까지 작업 반복
while current_date_WHILE <=end_date :
    print(current_date_WHILE)
    
    '''
    2. current_date 를 기반으로 날짜 변수를 datetime 형태로 저장  -
    ->    이후의 날짜 계산을 용이하게 하기 위함
    '''
    s_date = dt.datetime(current_date[0], current_date[1], current_date[2]) 
    e_date = s_date + dt.timedelta(days=1)   

    '''
    3. datetime 날짜의 type 조작   -->    검색창에 입력할 수 있는 형태로 변환
    '''

    # 1.  20200201  형태로 저장
    str_s_date1 = s_date.strftime('%Y%m%d')
    str_e_date1 = e_date.strftime('%Y%m%d')

    # 2,  2020.02.01 형태로 저장
    str_s_date2 = s_date.strftime('%Y.%m.%d')
    str_e_date2 = e_date.strftime('%Y.%m.%d')


    '''
    4. url 및 검색 parameter 지정
    '''

    # base url
    base_url = 'https://search.naver.com/search.naver'

    # parameters
    query_params = '?' + urlencode(
        {
            quote_plus('where') : 'news',
            quote_plus('query') : "+코로나 창업 | 스타트업",
            quote_plus('sm') : 'tab_opt',
            quote_plus('') : '',
            quote_plus('sort')  : '2',
            quote_plus('photo') : '0',
            quote_plus('field') : '0',
            quote_plus('pd') : '3',
            quote_plus('ds') : str_s_date2,
            quote_plus('de') : str_e_date2,
            quote_plus('docid') : '',  
            quote_plus('related') : '0',
            quote_plus('mynews') : '1',       # 언론사 선택시 1 / default = 0 
            quote_plus('office_type') :  '1',  # default = 0 / 일간지 = 1 
            quote_plus('office_section_code') : '1',  # 언론사 선택하면 1, default = 0
            
            ########################
            ### 신문사 코드 입력 ###
            ########################
            
            quote_plus('news_office_checked') : '1032',  #경향 1032,  만일 없으면 공백으로 비워두어야 함
        }
    )


    # 첫번째 반복시행에서 article index는 1 고정
    first_article_index = 1


    ###############################  특정 날짜에서의 검색창 크롤링 ###########################
    #iteration_idx = 0
    while True:    

        try:
            #debug code
            #iteration_idx +=1
            #print('while 작업 횟수 : ', iteration_idx)

            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)
            
        except:
            t.sleep(20)

            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)

            
        t.sleep(0.3)     # IP ban 방지용
        

        
        '''
        6. page parsing
        '''
        
        # bs parsing
        page = bs(page, 'html.parser')

        # IS_EMPTY 는 검색결과가 있다면 [] 를 반환함. 
        # 만일 IS_EMPTY 가 []가 아니라면 검색결과가 없는 것이므로 break 해야 함.
        IS_EMPTY = page.find_all('div', 'api_noresult_wrap')
        if IS_EMPTY != []:
            break 

        # 필요 데이터 parsing (url, 날짜)
        url_resultset = page.find_all('div', 'news_info')   #  --> url
        news_date_resultset = page.find_all('div', 'info_group')  # --> 날짜,  둘다 parsing 후의 index는 같음



        '''
        7. 지면뉴스 오류 handling
        '''
        for idx in range(len(url_resultset)):

            # url
            target_url = url_resultset[idx].a['data-url']

            # 날짜
            if news_date_resultset[idx].i:
                IS_PAPERNEWS = True
            else:
                IS_PAPERNEWS = False

            if IS_PAPERNEWS ==False:
                target_date = news_date_resultset[idx].find('span', 'info').text

            else:
                try:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[1].text
                except:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[0].text


            # 데이터 저장, key = url  / value = date
            data_set[target_url] = target_date




        # 다음 검색 페이지로 넘어가기 위해 기사 index 조정
        first_article_index += 10


    # 반복문 break에 사용할 current_date 설정
    current_date = s_date
    current_date += dt.timedelta(days = 1)  # 하루 더하기
    current_date = current_date.strftime('%Y%m%d')  # 더한 날짜정보를 str으로 바꾸기

    # str으로 변환된 날짜정보를 연,월,일로 분리
    year = current_date[:4]
    month = current_date[4:6]
    day = current_date[6:]

    # 다음 iteration에 넘어갈 current_date 정보
    current_date = (int(year), int(month), int(day))


    # 날짜 while문 종료에 관한 정보
    current_date_WHILE = int(dt.datetime(current_date[0], current_date[1], current_date[2]).strftime('%Y%m%d'))
    
    
    
'''
데이터 내보내기 (url)
'''     
        
table = pd.DataFrame(data_set.keys(), data_set.values())
table = table.drop_duplicates()
table.to_csv('경향 url주소.csv', encoding = 'cp949')

t.sleep(2)


########################################
##### 2. 본문 크롤링               #####
########################################


# url 가져오기 (csv파일)
urls = list(pd.read_csv('경향 url주소.csv', encoding = 'cp949')['0'])



# 데이터가 들어갈 공간
data_set = {}


'''
data columns :
    date   / title  / body   / company  / url
    (날짜) / (제목) / (본문) / (언론사) / (url)

'''


#url 접속 후 크롤링


for idx in range(len(urls)):
    
    # 차단 막기 위해 잠시 휴식
    #t.sleep(0.3)
    
    # 초기화
    body = ''
    title = ''
    day = ''
    company = ''

    
   # 한겨레
    try:
        req = Request(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
        )
        t.sleep(0.3)
        page = urlopen(req)
        page = bs(page, 'html.parser')
    except:
        t.sleep(20)
        req = Request(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
        )
        t.sleep(0.3)
        page = urlopen(req)
        page = bs(page, 'html.parser')
                               
    ### 데이터 저장 ###

    body = page.find_all('div', 'text')[0].text
    title = page.find("meta", {"name":"title"}).get("content")
    # title = page.find_all('title')[0].text
    day = page.find("meta", {"name":"publish"}).get("content")[:8]
    company = '경향'
    data_set[idx] = [day, title, body, company, urls[idx]]


        
'''
데이터 내보내기 (뉴스본문)
'''
        
dataframe = pd.DataFrame(data_set).transpose()
dataframe.columns = ['date', 'title', 'body', 'company', 'url']
# 본문 내 특수문자 제거
dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\xa0-\xff]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u1ecf]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u0001-\u0019]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u0021-\u9FBF]', repl = r'', regex=True)
dataframe['body']= dataframe['body'].str.replace(pat=r'[\u2026-\u2027]', repl = r'', regex=True)
dataframe['title']= dataframe['title'].str.replace(pat=r'[\u2026-\u2027]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(' b ', '')

dataframe = dataframe.reset_index()
dataframe.to_csv('경향{}to{}.csv'.format(first_date, end_date), encoding = "cp949")

20200201
20200202
20200203
20200204
20200205


KeyError: '0'

In [7]:
urls = list(pd.read_csv('한겨레 url주소.csv', encoding = 'cp949')['0'])
urls

['http://www.hani.co.kr/arti/opinion/column/926936.html',
 'http://www.hani.co.kr/arti/opinion/column/927023.html',
 'http://www.hani.co.kr/arti/science/future/927040.html']

In [8]:
url = 'https://www.hani.co.kr/arti/science/future/927040.html'
req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
)
t.sleep(0.3)
page = urlopen(req)
page = bs(page, 'html.parser')
page

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<!--[if IE]><![endif]-->
<!--[if lt IE 7 ]><html class="no-js ie6" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 7 ]><html class="no-js ie7" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 8 ]><html class="no-js ie8" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 9 ]><html class="no-js ie9" xmlns:fb="http://ogp.me/ns/fb#" lang="ko" xml:lang="ko"><![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="ko" xml:lang="ko" xmlns:fb="http://ogp.me/ns/fb#">
<!--<![endif]-->
<head>
<script type="text/javascript">
			window.googleDimension3	= decodeURIComponent('%EB%AF%B8%EB%9E%98%26%EA%B3%BC%ED%95%99');
		

In [9]:
url = 'https://www.hani.co.kr/arti/science/future/927040.html'
url

'https://www.hani.co.kr/arti/science/future/927040.html'

In [35]:
page.find_all('div', 'text')[0].text


'\n\n\n\n\n4일 중국 후베이성 우한의 신종 코로나바이러스 감염증 환자들을 수용해 임시 치료시설로 쓰던 3곳 중 하나인 전시관에서, 일꾼들이 전날 신축된 대형 응급 의료시설인 훠선산 병원으로 옮겨질 환자용 침대들을 한데 모아놓고 있다. 우한 시당국은 이 전시관을 비롯해 체육관과 복합문화센터 등 기존 수용시설에 있던 모든 환자를 신축 병원으로 옮겨 집중 치료할 계획이다. 우한/AFP 연합뉴스\n\n\n\n2020년 시작하자마자 전세계를 휩쓸고 있는 ‘2019 신종 코로나바이러스’ 감염증은 인류 역사에서 낯익은 전염병의 모습을 띠고 있지만, 기존의 감염질환과 구별되는 뚜렷한 속성들도 지니고 있다.\n세계 주요 거대도시를 빠르게 연결하는 항공편과 고속열차로 세상은 과거 어느 시점보다 더 연결됐고, 대도시의 집단 밀집주거와 대규모 상업시설로 인해 사람들의 주거 환경은 감염성 질환에 더욱 취약해졌다. 이와 더불어 전세계 대부분의 사람들이 스마트폰과 소셜미디어를 통해서 실시간으로 전염병 정보를 주고받는 상황은 일찍이 인류가 경험못한 상황이다. 인공지능 초연결 환경에서의 전염병 풍경은 무엇이 달라졌을까. \n1. 인공지능이 더 빠른 전염병 탐지\n신종 코로나바이러스의 확산을 가장 먼저 예측한 곳은 인공지능이었다. 지난해 12월31일 캐나다의 감염질환 예측 전문 신생기업 블루닷(BlueDot)은 ‘바이러스가 확산될 것’이라는 보고서를 내고 신종 코로나바이러스의 대유행을 예측했다. 이후 지난달 6일과 9일 각각 미국의 질병통제센터와 세계보건기구가 질병 확산을 공식 경고했다. 블루닷은 2003년 중국에서 발생한 사스(SARS·중증급성호흡기증후군)로 인해 캐나다인 44명이 숨진 것을 계기로, 토론토 최대 병원인 세인트마이클 병원의 임상의사였던 캄란 칸 박사가 창업한 기업이다. 블루닷은 인공지능으로 65개국의 뉴스를 비롯해 가축·동물 데이터, 모기 등 해충 현황, 국제 항공 이동 데이터 등을 수집해 질병 확산을 예측한다. 소셜미디어 데이터는 신뢰도가 떨어진다는 이유로 반영하

In [32]:
text = page.find_all('p', align="justify")
for p in text:
    print(p.string)

# bs.find_all('p', align="justify").text

None
None
None
None
None
None
None
None
None
None
None
None
None
